In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingRegressor, VotingRegressor, BaggingRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [54]:
!pip install lightgbm

In [3]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [1]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
import pandas as pd

In [7]:
file_paths = {
    "AlGr1.5": "C:\\Users\\Subhayu Kabiraj\\OneDrive\\Desktop\\Prediction of Field Emission Performance of MMC in UHV systems\\Stage_2\\stacking_predictions\\Stacking_Predicted_AlGr1.5.xlsx",
    "AlGr1.25": "C:\\Users\\Subhayu Kabiraj\\OneDrive\\Desktop\\Prediction of Field Emission Performance of MMC in UHV systems\\Stage_2\\stacking_predictions\\Stacking_Predicted_AlGr1.5.xlsx",
    "AlGr1.75": "C:\\Users\\Subhayu Kabiraj\\OneDrive\\Desktop\\Prediction of Field Emission Performance of MMC in UHV systems\\Stage_2\\stacking_predictions\\Stacking_Predicted_AlGr1.5.xlsx",
    "AlGr2": "C:\\Users\\Subhayu Kabiraj\\OneDrive\\Desktop\\Prediction of Field Emission Performance of MMC in UHV systems\\Stage_2\\stacking_predictions\\Stacking_Predicted_AlGr1.5.xlsx"
}
file_path = "stage2_model_metrics.xlsx"

In [26]:
data_frames = [pd.read_excel(path, sheet_name="Sheet1") for path in file_paths.values()]
combined_data = pd.concat(data_frames, ignore_index=True)
df = pd.read_excel(file_path)

In [8]:
X = combined_data[["Time (Minutes)", "Electric Field Strength"]]
y_current = combined_data["Predicted Current (microAmperes)"]
y_density = combined_data["Current Density"]


In [9]:
poly = PolynomialFeatures(degree=3, include_bias=False)
X_poly = poly.fit_transform(X)

In [10]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_poly)


In [11]:
X_train, X_test, y_current_train, y_current_test = train_test_split(X, y_current, test_size=0.2, random_state=42)
X_train, X_test, y_density_train, y_density_test = train_test_split(X, y_density, test_size=0.2, random_state=42)

In [12]:
xgb = XGBRegressor(n_estimators=500, learning_rate=0.01, max_depth=7, subsample=0.95, colsample_bytree=0.95, random_state=42)
lgb = LGBMRegressor(n_estimators=500, learning_rate=0.01, max_depth=7, num_leaves=31, subsample=0.95, colsample_bytree=0.95, random_state=42)
cat = CatBoostRegressor(iterations=500, learning_rate=0.01, depth=7, l2_leaf_reg=3, verbose=0, random_state=42)

In [13]:
xgb.fit(X_train, y_current_train)
lgb.fit(X_train, y_current_train)
cat.fit(X_train, y_current_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000268 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 144
[LightGBM] [Info] Number of data points in the train set: 233, number of used features: 2
[LightGBM] [Info] Start training from score 1.147745
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [23]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    adj_r2 = 1 - (1 - r2) * ((len(y_test) - 1) / (len(y_test) - X_test.shape[1] - 1))
    return {"MSE": mse, "RMSE": rmse, "R2": r2, "Adjusted R2": adj_r2}
print(len(y_current_test))
print(X_test.shape[1])

59
2


In [19]:
stacking = StackingRegressor(
    estimators=[("xgb", xgb), ("lgb", lgb), ("cat", cat)], final_estimator=Ridge(alpha=0.5)
)
stacking.fit(X_train, y_current_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000027 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 144
[LightGBM] [Info] Number of data points in the train set: 233, number of used features: 2
[LightGBM] [Info] Start training from score 1.147745
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

StackingRegressor(estimators=[('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.95, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_ra...
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=500, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...)),
                              ('lgb',
                               LGBMRegressor(colsample_bytree=0.95,
                                             learning_rate=0.01, max_depth=7,
                                             n_estimators=500, random_state=42,
                                             subsample=0.95)),
                              ('cat',
                               <catboost.core.CatBoostRegressor object at 0x000002298E18B290>)],
                  final_estimator=Ridge(alpha=0.5))

In [20]:
voting = VotingRegressor([("xgb", xgb), ("lgb", lgb), ("cat", cat)], weights=[0.3, 0.3, 0.4])
voting.fit(X_train, y_current_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000024 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 144
[LightGBM] [Info] Number of data points in the train set: 233, number of used features: 2
[LightGBM] [Info] Start training from score 1.147745
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

VotingRegressor(estimators=[('xgb',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=0.95, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate...
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None, n_estimators=500,
                                          n_jobs=None, num_parallel_tree=None,
                                          random_state=42, ...)),
                            ('lgb',
                             LGBMRegressor(colsample_bytree=0.95,
                                           learning_rate=0.01, max_depth=7,
                                           n_estimators=500, random_state=42,
                                           subsample=0.95)),
                            ('cat',
                             <catboost.core.CatBoostRegressor object at 0x000002298E18B290>)],
                weights=[0.3, 0.3, 0.4])

In [21]:
bagging = BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=7), n_estimators=50, random_state=42)
bagging.fit(X_train, y_current_train)


C:\Users\Subhayu Kabiraj\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=7),
                 n_estimators=50, random_state=42)

In [22]:
results = {
    "XGBoost": evaluate_model(xgb, X_test, y_current_test),
    "LightGBM": evaluate_model(lgb, X_test, y_current_test),
    "CatBoost": evaluate_model(cat, X_test, y_current_test),
    "Stacking": evaluate_model(stacking, X_test, y_current_test),
    "Voting": evaluate_model(voting, X_test, y_current_test),
    "Bagging": evaluate_model(bagging, X_test, y_current_test)
}


In [27]:
print("\t\t\tFinal Model Performance\n")
print(df.to_string(index=False))

			Final Model Performance

              Model        MSE      RMSE       R2   Adj R2
           Stacking 155.846027 12.483831 0.933962 0.931604
       CatBoost_GPR 167.509048 12.942529 0.921666 0.918869
        XGBoost_GPR 174.893597 13.224734 0.915848 0.912842
       CatBoost_SVR 182.671739 13.515611 0.902046 0.898548
       LightGBM_GPR 180.199228 13.423831 0.901962 0.898461
        XGBoost_SVR 190.843699 13.814619 0.891065 0.887175
CatBoost_Polynomial 224.542261 14.984734 0.885848 0.881771
       LightGBM_SVR 195.492670 13.981869 0.884140 0.880003
        CatBoost_RF 205.134827 14.322529 0.881666 0.877440
         XGBoost_RF 214.201108 14.635611 0.881046 0.876798
             Voting 197.535555 14.054734 0.875848 0.871414
 XGBoost_Polynomial 230.876448 15.194619 0.871065 0.866461
        LightGBM_RF 219.449577 14.813831 0.870962 0.866354
         Bagging_RF 245.294150 15.661869 0.854140 0.848931
LightGBM_Polynomial 236.622188 15.382529 0.851666 0.846369
        Bagging_SVR 219.2061